# Permutations of a Simple Circuit

This notebook walks through how to utilize the core semantics of SysML v2 to generate alternative circuits as inputs to an OpenMDAO solution of these circuits.

If you want to play with a widget that does this, just run the cell below, if you want to understand the rest of the process, follow the rest of the notebook.

In [ ]:
import sys
from pathlib import Path

import pymbe.api as pm

pymbe_tests = (Path(pm.__file__).parent / "../../tests").resolve().absolute()
assert pymbe_tests.is_dir(), "Cannot find pymbe tests folder!"

if str(pymbe_tests) not in sys.path:
    sys.path.append(str(pymbe_tests))
    
from interpretation.circuit_example import CircuitComponent, CircuitUI

ui = CircuitUI()
ui

## Background

The M1 user model in SysML v2 is meant to be a set of constraints and rules under which legal instances can be created. Those instances should be taken as alternative produced systems and they can be analyzed in that way.

## Load Up Model

Read the model from the local JSON file.

In [ ]:
circuit_file = pymbe_tests / "fixtures/Circuit Builder.json"

circuit_model = pm.Model.load_from_file(circuit_file)
circuit_model.max_multiplicity = 100

## Explore Contents of Model with M1 in Memory

Use the M1 memory objects to see what is in the current model, starting with the main packages.

In [ ]:
circuit_model.packages

In [ ]:
circuit_model.ownedElement["Circuit Builder"].ownedElement

In [ ]:
circuit_def = circuit_model.ownedElement["Circuit Builder"].ownedElement["Circuit"]

### Circuit and its Features

Here is the circuit and its features, both parts and used connections.

In [ ]:
circuit_def.relationships

In [ ]:
circuit_def.ownedMember

## Update multiplicities
for `Resistors`, `Diodes`, and `Connections`

## Generate M0 instances from the M1 model

Use the M1 model to start creating a series of instances to represent the circuits that should be analyzed.

In [ ]:
NUM_INTERPRETATIONS = 10

m0_interpretations = [
    pm.random_generator_playbook(
        m1=circuit_model,
        filtered_feat_packages=[circuit_model.ownedElement["Circuit Builder"]],
    ) for _ in range(NUM_INTERPRETATIONS)
]

### Sort the interpretations by number of connections
Sorted from `most` to `least`, and pick the first one.

In [ ]:
circuit_model.elements["9cf7b7c6-194a-4882-8480-05a807d1391f"]

In [ ]:
cf = circuit_model.ownedElement["Circuit Builder"].ownedElement["Circuit"]
#cf.relationships["reverseFeatureTyping"][0].owner
cf.throughFeatureMembership

In [ ]:
# sort the interpretations from most connections to fewer connections
for member in circuit_def.ownedMember:
    if member._metatype == "ConnectionUsage":
        connection = member
        break

m0_interpretations = [*sorted(m0_interpretations, key=lambda x: len(x[connection._id]), reverse=True)]
m0_interpretation = m0_interpretations[0]

## Filter M0 Instances for Reasonable Circuits

Until we get more sophisticated and can interpret constraints, the initial approach is to filter out solutions with unanalyzable layouts or trim the layouts to something more tractable.

### Connector End Checks

Look at the ends of the three main kinds of connectors.

In [ ]:
p2p = circuit_def.ownedMember["Part to Part"]
p2p.endFeature[0]._id

In [ ]:
source_feat, target_feat = p2p.endFeature
for source, target in zip(m0_interpretation[source_feat._id], m0_interpretation[target_feat._id]):
    print(source, "-->", target)

# OpenMDAO
> Based on OpenMDAO's [nonlinear circuit analysis example](https://openmdao.org/newdocs/versions/latest/examples/circuit_analysis_examples.html).

In [ ]:
from importlib import import_module

import networkx as nx
import openmdao.api as om

In [ ]:
problem = ui.parametric_executor.problem

if problem:
    om.view_connections(problem)
else:
    print("Click the '+' button in the dashboard until it generates a circuit")

In [ ]:
if problem:
    om.n2(problem)
else:
    print("Click the '+' button in the dashboard until it generates a circuit")